In [ ]:
import sys
sys.path.append("..")
sys.path.append("my_library")
from my_library.config import *
from my_library.metrics.cmp_metrics import *
sys.path.append("..")
import os

import os


In [ ]:
'''ArithmeticErrormodels = ["br3", "adsq", "pdl1-3"] 
outcomes = ["BEST_RESPONSE_3", "HISTOLOGY", "PDL1_CATHEGORY"]
eval = "eval_1_single_fold"'''
'''
base_folder = os.path.join(working_directory,"eval")
subfolder = [name for name in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, name))]

results = []

eval = "eval"

models = ["br3", "adsq", "pdl1-3"] 
outcomes = ["BEST_RESPONSE_3", "HISTOLOGY", "PDL1_CATHEGORY"]

for m,o in zip(models, outcomes):
    print(m)
    print(o)

    outcome = o
    model = m
    root_dir = os.path.join(base_folder, m)'''

In [ ]:
import pandas as pd
csv_df = pd.read_csv(working_directory+"/projects/I3lung-sqadqc-project/annotations.csv")


# setup check

In [ ]:
csv_df

# Find all the paths

In [ ]:
import os

root_dir = f"{working_directory}/{eval}/train/" + model

train_parquet_paths = []
val_parquet_paths = []
test_parquet_paths = []
all_parquets_paths = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                train_parquet_paths.append(file_path)
train_parquet_paths.sort()

#print("Sorted Parquet paths:")
#for path in train_parquet_paths:
#    print(path)

root_dir = f"{working_directory}/{eval}/val/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                val_parquet_paths.append(file_path)
val_parquet_paths.sort()


root_dir = f"{working_directory}/{eval}/test-int/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                test_parquet_paths.append(file_path)
test_parquet_paths.sort()

root_dir = f"{working_directory}/{eval}/all/int_folds_{model}"

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                all_parquets_paths.append(file_path)
all_parquets_paths.sort()


In [ ]:
print(all_parquets_paths)

# Train Dummy models

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, accuracy_score

In [ ]:
print(all_parquets_paths[0])

In [ ]:
metrics_list = []
numbers = []

test_scores = []
test_true_y = []
test_pred_y =[]

test_list_dataframes = []




path = all_parquets_paths[0]

df_predictions = pd.read_parquet(path)
df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
#print(df_joined)
distribution = df_joined[["slide", outcome]]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_list = np.ones((distribution.shape[0], 1))
y_list = df_joined[outcome].to_numpy()

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_list, y_list)

y_pred = dummy_clf.predict(x_list)



df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
#print(df_joined)
distribution = df_joined[["slide",outcome]]

display(distribution)

y_true = df_joined[outcome].to_numpy()
print(y_true)
x_list = np.ones((distribution.shape[0], 1))



#print(distribution)
y_pred = dummy_clf.predict(x_list)
print(y_pred)

test_true_y.append(y_true)
test_pred_y.append(y_pred)

distribution["y_pred"] = y_pred
test_list_dataframes.append(distribution)




In [ ]:
print(len(test_list_dataframes))

In [ ]:
test_df = pd.concat(test_list_dataframes, ignore_index = True)
test_df.rename(columns={test_df.columns[1]: 'y_true'}, inplace=True)

In [ ]:
display(test_df)

In [ ]:
distribution

In [ ]:
y_true

In [ ]:
def extract_5_elements (train_true_y, train_pred_y):
    liste =train_true_y
    y_trues = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_trues.append(unione)
        #print(unione)

    liste = train_pred_y
    y_preds = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_preds.append(unione)
        #print(unione)
    return y_trues, y_preds


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def average(df_input, name): 
    df = df_input.copy()
    # Removing "Fold" column
    df = df.drop(columns=["Dataset","Fold"])  
    mean_values = df.mean()
    std_values = df.std()
    n = len(df)
    ci_values = (2 * std_values) / np.sqrt(n)
    result_df = pd.DataFrame({
    "Column": mean_values.index,
    "Mean": mean_values.values,
    "CI": ci_values.values
    })
    result_df = result_df.add_prefix(name)
    display(result_df)
    return result_df

In [ ]:
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    y_trues, y_preds = extract_5_elements(train_true_y, train_pred_y)
    results = []  # Store results

    for i in range(len(y_trues)):
        y_list = y_trues[i]
        y_pred = y_preds[i]
        acc = accuracy_score(y_list, y_pred)
        precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
        recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

        report = classification_report(y_list, y_pred, digits=2, output_dict=True, zero_division=0)

        # Flatten report into a list
        row = {"Dataset": name_set, "Fold": i+1, "Accuracy": acc, "Precision" : precision, "Recall": recall, "F1_score" : f1}

        for label, metrics in report.items():
            if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
                for metric_name, value in metrics.items():
                    row[f"{label}_{metric_name}"] = value
        
        results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    avg = average(df_results, name_set)

    return df_results, avg
    


In [ ]:
#df1, avg_1 = extract_metrics( train_true_y, train_pred_y, model, "train")
#df2, avg_2= extract_metrics( val_true_y, val_pred_y, model, "val")
df3, avg_3 = extract_metrics( test_true_y, test_pred_y, model, "test")

#df_final = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
#df_final.to_excel(model+f"_complete_5_folds.xlsx")

#df_final_avg = pd.concat([avg_1, avg_2, avg_3], axis=1, ignore_index=False)
#df_final_avg.to_excel(model+f"_complete_averages_5_folds.xlsx")

In [ ]:
from unittest import TestCase
test_df

In [ ]:
# Raggruppiamo i dati per slide e y_true e creiamo nuove colonne per ogni valore di y_pred_max_idx
df_final = test_df.groupby(["slide", "y_true"])["y_pred"].agg(list).reset_index()

# Espandiamo la lista in colonne separate
df_final_expanded = df_final["y_pred"].apply(pd.Series)

# Rinominiamo le nuove colonne
df_final_expanded.columns = [f"pred_{i+1}" for i in df_final_expanded.columns]

# Concatenazione con le colonne originali
df_result = pd.concat([df_final.drop(columns=["y_pred"]), df_final_expanded], axis=1)

In [ ]:
df_result

In [ ]:
a = [col for col in df_result.columns if 'pred_' in col]

In [ ]:

# Supponiamo che il tuo DataFrame si chiami df
colonne_pred = [col for col in df_result.columns if 'pred_' in col]#[f"pred_{i}" for i in range(1, 6)]

# Calcola il valore più frequente (majority vote) per ogni riga
df_result['y_pred'] = df_result[colonne_pred].mode(axis=1)[0]

# Visualizza il DataFrame aggiornato
print(df_result.head())

In [ ]:
df_test = df_result[["slide", "y_true", "y_pred"]]

In [ ]:
df_test

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    
    results = []  # Store results
    y_list = train_true_y
    y_pred = train_pred_y
    acc = accuracy_score(y_list, y_pred)

    precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
    weighted_accuracy = balanced_accuracy_score(y_list, y_pred)

    recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

    report = classification_report(y_list, y_pred, digits=2,   output_dict=True, zero_division=0)

    # Flatten report into a list
    row = {"Dataset": name_set, "Fold": 1, "Accuracy": acc, "Balanced Accuracy":weighted_accuracy, "Precision" : precision, "Recall": recall, "F1_score" : f1}

    for label, metrics in report.items():
        if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
            for metric_name, value in metrics.items():
                row[f"{label}_{metric_name}"] = value
        
    results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    

    return df_results

In [ ]:
y_true = df_test["y_true"].to_numpy()
y_pred = df_test["y_pred"].to_numpy()

In [ ]:
df = extract_metrics(y_true, y_pred, "br3", "test")